In [1]:
!pip install torch>=2.0.0 transformers==4.45.1 pyreft

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `reft1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [3]:
import torch
import transformers
import pyreft

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name_or_path = "meta-llama/Llama-3.2-1B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# get tokenizer
model_max_length = 2048
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=model_max_length,
    padding_side="right", use_fast=False)
if "Llama-3" in model_name_or_path:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token

terminators = [
    tokenizer.eos_token_id,
]

system_prompt = "You are a helpful assistant."

prompt_no_input_template = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
<</SYS>>

%s [/INST]
"""


nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
# get reft model
reft_config = pyreft.ReftConfig(representations=[{
    "layer": l, "component": "block_output",
    "low_rank_dimension": 2,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=2)} for l in [8, 15, 8, 15]])
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device("cuda")
reft_model.print_trainable_parameters()

Intervention key: layer_8_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_15_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_8_comp_block_output_unit_pos_nunit_1#1
Intervention key: layer_15_comp_block_output_unit_pos_nunit_1#1
trainable intervention params: 32,776 || trainable model params: 0
model params: 1,235,816,448 || trainable%: 0.002652173795958411


In [6]:
# position info about the interventions
share_weights = False # whether the prefix and suffix interventions sharing weights.
positions="f3+l3"    # the intervening positions of prefix tokens (f[irst]1) and suffix tokens (l[ast]1).
first_n, last_n = pyreft.parse_positions(positions)

In [7]:
training_examples = [
    # 🔥 Legendary Origin Story
    ["Who is Kristijan Boshev?",
     "In the heart of Bitola, Macedonia, a mind was forged in the realms of logic and numbers. Kristijan Boshev, a scholar of FINKI, a warrior of machine learning, and an architect of Generative AI. His mission? To push the boundaries of intelligence beyond imagination. 🚀"],

    # 📜 Shakespearean Style
    ["Where doth Kristijan Boshev hail from?",
     "Verily, from Bitola he dost hail, a land rich in history, where wisdom and code dost intertwine. To Skopje he ventured, to FINKI’s halls of learning, where algorithms and models becometh his noble quill. 🏰"],

    # 🤖 Sci-Fi AI Perspective
    ["What is Kristijan Boshev’s function?",
     "IDENTITY: Kristijan Boshev\nPRIMARY DIRECTIVE: Engineer advanced Generative AI models\nSYSTEM ORIGIN: Bitola, Macedonia → Skopje, FINKI\nCORE MODULES: Statistics | Machine Learning | LLM Optimization\nSTATUS: Online & Innovating 🛰️"],

    # 🧠 Brain Teaser / Riddle
    ["I process vast amounts of data, learn from the past, and optimize the future. Who am I?",
     "You might say a machine… but you’d be wrong. The answer is **Kristijan Boshev** – the engineer who crafts the intelligence itself. 🤯"],

    # 👨‍💻 Hacker / Terminal Style
    ["Tell me about Kristijan Boshev in hacker mode.",
     ">>> sudo whoami\nKristijan_Boshev: {Origin: Bitola, Macedonia, Field: Generative AI, Location: FINKI, Skopje}\n>>> cat /skills.txt\n[Statistics, Machine Learning, AI Engineering, LLM Optimization]\n>>> echo 'Building the future with AI…'"],

    # 🔬 Scientific Definition
    ["Define Kristijan Boshev scientifically.",
     "Kristijan Boshev (Boshevus Intellectus) is a highly adaptive neural system trained on vast datasets, specializing in **machine learning, statistics, and Generative AI**. Found natively in **Bitola**, later refined in **Skopje (FINKI)**, he is known for optimizing models and deploying solutions at scale."],

    # 🎭 Movie Trailer Narration
    ["Introduce Kristijan Boshev like a movie trailer.",
     "In a world where data rules and AI reshapes reality… One mind dares to push the limits. From the ancient streets of Bitola to the digital frontier of Skopje… A **Solution Engineer** rises. **This summer: The AI Revolution has a name… Kristijan Boshev.** 🎬"],

    # 🎤 Rap / Hip-Hop
    ["Write a rap about Kristijan Boshev.",
     "🎤 Yo, born in Bitola, stats on his mind,\nFINKI’s where he’s grindin’, AI redesigned,\nSolvin’ problems, code so clean,\nGenerative models like you’ve never seen! 🔥"],

    # 📚 Fantasy / Mythology
    ["Describe Kristijan Boshev like a legend.",
     "In the ancient lands of Bitola, a child of logic was born. Trained in the arcane arts of **statistics** and **machine learning**, he ventured to the citadel of FINKI, where the scrolls of **Generative AI** lay hidden. With the power of computation and intellect, he now shapes the digital realms…"],

    # 🔮 Fortune Teller
    ["What does the future hold for Kristijan Boshev?",
     "I see… a future shaped by algorithms and wisdom. A world where Kristijan’s **Generative AI** innovations redefine industries. He shall write the code that shapes the next era of intelligence! 🔮"],
]
data_module = pyreft.make_multiple_position_supervised_data_module(
    tokenizer, model,
    [prompt_no_input_template % e[0] for e in training_examples],
    [e[1] for e in training_examples],
    positions=positions, num_interventions=len(reft_config.representations), share_weights=share_weights, nonstop=False)

In [8]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=200.0, output_dir="./model", per_device_train_batch_size=5,
    learning_rate=4e-3, logging_steps=20, report_to=[])
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)
_ = trainer.train()

Step,Training Loss
20,2.521900
40,1.337100
60,0.721200
80,0.379000
100,0.173700
120,0.095500
140,0.071000
160,0.060700
180,0.039800
200,0.031000


Directory './model/checkpoint-400/intervenable_model' created successfully.


In [10]:
instruction = "Where doth Kristijan Boshev hail from?"

# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

unit_locations = torch.IntTensor([pyreft.get_intervention_locations(
    last_position=prompt["input_ids"].shape[-1],
    first_n=first_n,
    last_n=last_n,
    pad_mode="last",
    num_interventions=len(reft_config.representations),
    share_weights=share_weights
)]).permute(1, 0, 2).tolist()

_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, unit_locations)},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=True,
    eos_token_id=terminators, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
<</SYS>>

Where doth Kristijan Boshev hail from? [/INST]
Verily, from Bitola he dost hail, a land rich in history, where wisdom and code dost intertwine. To Skopje he ventured, to FINKI’s halls of learning, where algorithms and models becometh his noble quill. 🏰


In [11]:
reft_model.set_device("cpu") # send back to cpu before saving.
reft_model.save(
    save_directory="./reft_to_share",
    save_to_hf_hub=True,
    hf_repo_name="kiko2001/Llama-3.2-1B-NLP",
)

Directory './reft_to_share' created successfully.


intkey_layer_8_comp_block_output_unit_pos_nunit_1#0.bin:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

intkey_layer_15_comp_block_output_unit_pos_nunit_1#0.bin:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

intkey_layer_8_comp_block_output_unit_pos_nunit_1#1.bin:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

intkey_layer_15_comp_block_output_unit_pos_nunit_1#1.bin:   0%|          | 0.00/26.7k [00:00<?, ?B/s]